In [2]:
import pandas as pd
# Устанавливаем отображение всех столбцов
pd.set_option('display.max_columns', None)

!pip install lightfm

In [3]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
events_df = pd.read_csv('events.csv')

In [5]:
item_features_df = pd.read_csv('item_features.csv')

In [6]:
submission_sample_df = pd.read_csv('submission_sample.csv')
submission_sample_df 


,user_id,item_id
0,0,0 1 2 3 4 5 6 7 8 9
1,1,0 1 2 3 4 5 6 7 8 9
2,2,0 1 2 3 4 5 6 7 8 9
3,3,0 1 2 3 4 5 6 7 8 9
4,4,0 1 2 3 4 5 6 7 8 9
...,...,...
6035,6035,0 1 2 3 4 5 6 7 8 9
6036,6036,0 1 2 3 4 5 6 7 8 9
6037,6037,0 1 2 3 4 5 6 7 8 9
6038,6038,0 1 2 3 4 5 6 7 8 9


In [7]:
user_features_df = pd.read_csv('user_features.csv')
user_features_df

,user_id,gender,age
0,4855,F,1
1,4065,M,56
2,3331,M,25
3,5373,M,45
4,2032,M,25
...,...,...,...
6035,4455,F,25
6036,5002,F,45
6037,4365,F,56
6038,3700,F,45


In [8]:
def analyze_dataframes(dataframes):
    for i, df in enumerate(dataframes):
        print(f"DataFrame {i+1}:")
        
        # Выводим информацию о датафрейме
        print("\nИнформация о датафрейме:")
        print(df.info())
        
        # Выводим названия столбцов
        print("\nНазвания столбцов:")
        print(df.columns.tolist())
        
        # Выводим минимальные и максимальные значения для числовых столбцов
        print("\nМинимальные и максимальные значения:")
        print(df.min())
        print(df.max())
        
        # Выводим дубликаты
        duplicates = df.duplicated().sum()
        print(f"\nКоличество дубликатов: {duplicates}")
        
        # Выводим основные статистические характеристики
        print("\nСтатистические характеристики:")
        print(df.describe(include='all'))
        
        # Выводим уникальные значения в каждом столбце
        for column in df.columns:
            unique_values = df[column].unique()
            print(f"\nУникальные значения в столбце '{column}':")
            print(unique_values)
        
        print("|" + "="*50 + "|")

In [9]:
analyze_dataframes([events_df])

DataFrame 1:

Информация о датафрейме:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 894149 entries, 0 to 894148
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    894149 non-null  int64
 1   item_id    894149 non-null  int64
 2   rating     894149 non-null  int64
 3   timestamp  894149 non-null  int64
dtypes: int64(4)
memory usage: 27.3 MB
None

Названия столбцов:
['user_id', 'item_id', 'rating', 'timestamp']

Минимальные и максимальные значения:
user_id      0
item_id      0
rating       1
timestamp    0
dtype: int64
user_id      6039
item_id      3705
rating          5
timestamp    2312
dtype: int64

Количество дубликатов: 0

Статистические характеристики:
             user_id        item_id         rating      timestamp
count  894149.000000  894149.000000  894149.000000  894149.000000
mean     3000.009516    1844.262022       3.580947     194.684738
std      1732.316153    1077.520786       1.116696     

In [10]:
analyze_dataframes([user_features_df])  

DataFrame 1:

Информация о датафрейме:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  6040 non-null   int64 
 1   gender   6040 non-null   object
 2   age      6040 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 141.7+ KB
None

Названия столбцов:
['user_id', 'gender', 'age']

Минимальные и максимальные значения:
user_id    0
gender     F
age        1
dtype: object
user_id    6039
gender        M
age          56
dtype: object

Количество дубликатов: 0

Статистические характеристики:
            user_id gender          age
count   6040.000000   6040  6040.000000
unique          NaN      2          NaN
top             NaN      M          NaN
freq            NaN   4331          NaN
mean    3019.500000    NaN    30.639238
std     1743.742145    NaN    12.895962
min        0.000000    NaN     1.000000
25%     1509.750000    NaN    25.

In [11]:
analyze_dataframes([item_features_df])  

DataFrame 1:

Информация о датафрейме:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3706 entries, 0 to 3705
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   item_id   3706 non-null   int64
 1   genre_0   3706 non-null   int64
 2   genre_1   3706 non-null   int64
 3   genre_2   3706 non-null   int64
 4   genre_3   3706 non-null   int64
 5   genre_4   3706 non-null   int64
 6   genre_5   3706 non-null   int64
 7   genre_6   3706 non-null   int64
 8   genre_7   3706 non-null   int64
 9   genre_8   3706 non-null   int64
 10  genre_9   3706 non-null   int64
 11  genre_10  3706 non-null   int64
 12  genre_11  3706 non-null   int64
 13  genre_12  3706 non-null   int64
 14  genre_13  3706 non-null   int64
 15  genre_14  3706 non-null   int64
 16  genre_15  3706 non-null   int64
 17  genre_16  3706 non-null   int64
 18  genre_17  3706 non-null   int64
dtypes: int64(19)
memory usage: 550.2 KB
None

Названия столбцов:
['item_id',

In [28]:
df = user_features_df.merge(events_df, on="user_id", how="left").merge(item_features_df, on="item_id", how="left")

In [29]:
df

,user_id,gender,age,item_id,rating,timestamp,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17
0,4855,F,1,3253,4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,4855,F,1,3367,4,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
2,4855,F,1,2732,5,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
3,4855,F,1,2587,5,3,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,4855,F,1,3608,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894144,4863,M,25,1889,5,335,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
894145,4863,M,25,1140,4,336,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
894146,4863,M,25,3286,4,337,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
894147,4863,M,25,3402,3,338,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0


In [30]:
unique_values = df['user_id'].unique()
len(unique_values)

6040

In [31]:
df[df['user_id']==0].sort_values(by='timestamp')

,user_id,gender,age,item_id,rating,timestamp,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17
109412,0,M,35,1505,4,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
109413,0,M,35,3669,3,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
109414,0,M,35,584,4,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
109415,0,M,35,3390,3,3,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
109416,0,M,35,2885,4,4,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109694,0,M,35,1124,3,321,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
109695,0,M,35,1809,4,322,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
109696,0,M,35,3602,4,323,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0
109697,0,M,35,2924,3,325,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [32]:
# Разбиваем на возрастные группы
bins = [0, 25, 45, 65, 100]
labels = ['<25', '25-45', '45-65', '65-100']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

In [33]:
df

,user_id,gender,age,item_id,rating,timestamp,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17,age_group
0,4855,F,1,3253,4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,<25
1,4855,F,1,3367,4,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,<25
2,4855,F,1,2732,5,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,<25
3,4855,F,1,2587,5,3,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,<25
4,4855,F,1,3608,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,<25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894144,4863,M,25,1889,5,335,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,25-45
894145,4863,M,25,1140,4,336,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,25-45
894146,4863,M,25,3286,4,337,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,25-45
894147,4863,M,25,3402,3,338,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,25-45


In [34]:
# Количество фильмов, просмотренных пользователем в каждом жанре
genre_columns = [f'genre_{i}' for i in range(18)]
df['total_genres_watched'] = df[genre_columns].sum(axis=1)

In [35]:
df

,user_id,gender,age,item_id,rating,timestamp,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17,age_group,total_genres_watched
0,4855,F,1,3253,4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,<25,1
1,4855,F,1,3367,4,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,<25,2
2,4855,F,1,2732,5,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,<25,2
3,4855,F,1,2587,5,3,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,<25,3
4,4855,F,1,3608,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,<25,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894144,4863,M,25,1889,5,335,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,25-45,2
894145,4863,M,25,1140,4,336,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,25-45,2
894146,4863,M,25,3286,4,337,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,25-45,2
894147,4863,M,25,3402,3,338,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,25-45,2


In [36]:
user_avg_rating = df.groupby('user_id')['rating'].mean().reset_index()
user_avg_rating.columns = ['user_id', 'user_avg_rating']

df = df.merge(user_avg_rating, on='user_id')

In [37]:
df

,user_id,gender,age,item_id,rating,timestamp,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17,age_group,total_genres_watched,user_avg_rating
0,4855,F,1,3253,4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,<25,1,4.162791
1,4855,F,1,3367,4,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,<25,2,4.162791
2,4855,F,1,2732,5,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,<25,2,4.162791
3,4855,F,1,2587,5,3,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,<25,3,4.162791
4,4855,F,1,3608,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,<25,1,4.162791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894144,4863,M,25,1889,5,335,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,25-45,2,3.607383
894145,4863,M,25,1140,4,336,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,25-45,2,3.607383
894146,4863,M,25,3286,4,337,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,25-45,2,3.607383
894147,4863,M,25,3402,3,338,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,25-45,2,3.607383


In [38]:
df['genre_combination'] = df[genre_columns].astype(str).agg(''.join, axis=1)

In [39]:
gender_genre_pref = df.groupby('gender')[genre_columns].sum().reset_index()

In [40]:
gender_genre_pref

,gender,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17
0,F,40748,24372,10939,19090,85970,14732,1720,87655,7763,3730,12977,11992,8905,44876,24496,35907,12579,3113
1,M,189541,95453,27814,45574,232890,56593,5295,229551,24554,12558,55217,25187,26963,86934,116134,133546,48779,15385


In [41]:
# Считаем, сколько фильмов каждого жанра пользователь посмотрел
user_genre_interaction = df.groupby('user_id')[genre_columns].sum().reset_index()

In [42]:
user_genre_interaction

,user_id,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17
0,0,95,61,26,43,105,20,0,71,19,4,22,18,13,54,47,65,17,4
1,1,85,34,7,9,68,32,3,69,7,5,32,5,9,24,61,105,14,5
2,2,83,41,8,8,35,11,1,30,4,1,11,3,4,15,42,44,9,2
3,3,136,62,9,16,84,17,0,30,9,0,18,4,6,16,76,66,13,4
4,4,43,12,4,4,39,10,0,36,1,0,2,3,2,13,19,28,6,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,6035,40,19,4,4,13,9,0,33,4,2,6,1,0,5,50,19,11,0
6036,6036,5,3,1,1,9,4,0,16,2,1,1,1,1,3,4,4,4,1
6037,6037,14,8,3,2,36,5,4,36,0,3,5,4,4,10,8,12,5,1
6038,6038,146,117,61,123,300,29,1,164,44,8,37,54,21,79,103,67,30,13


In [43]:
# Считаем, сколько фильмов каждого жанра посмотрели пользователи в каждой возрастной группе
age_genre_pref = df.groupby('age_group')[genre_columns].sum().reset_index()

C:\Users\admin\AppData\Local\Temp\ipykernel_18048\3536820105.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_genre_pref = df.groupby('age_group')[genre_columns].sum().reset_index()


In [44]:
age_genre_pref

,age_group,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17
0,<25,50751,27086,11388,19061,72603,15267,1079,58644,8214,2317,15575,8229,6515,26072,29633,36268,11184,2894
1,25-45,139730,70921,22009,35601,190057,43017,4589,188126,18954,9608,41325,21003,20869,78110,85380,102163,35236,10371
2,45-65,39808,21818,5356,10002,56200,13041,1347,70436,5149,4363,11294,7947,8484,27628,25617,31022,14938,5233
3,65-100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [45]:
# Приджойним фичу gender_genre_pref на основе колонки 'gender'
df = df.merge(gender_genre_pref, on='gender', suffixes=('', '_gender_pref'))

# Приджойним фичу user_genre_interaction на основе колонки 'user_id'
df = df.merge(user_genre_interaction, on='user_id', suffixes=('', '_user_genre_interaction'))

# Приджойним фичу age_genre_pref на основе колонки 'age_group'
df = df.merge(age_genre_pref, on='age_group', suffixes=('', '_age_genre_pref'))

In [46]:
# Приджойним средний рейтинг фильма
item_avg_rating = df.groupby('item_id')['rating'].mean().reset_index()
item_avg_rating.columns = ['item_id', 'item_avg_rating']
df = df.merge(item_avg_rating, on='item_id')

In [47]:
unique_values = df['user_id'].unique()
len(unique_values)

6040

In [48]:
df

,user_id,gender,age,item_id,rating,timestamp,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17,age_group,total_genres_watched,user_avg_rating,genre_combination,genre_0_gender_pref,genre_1_gender_pref,genre_2_gender_pref,genre_3_gender_pref,genre_4_gender_pref,genre_5_gender_pref,genre_6_gender_pref,genre_7_gender_pref,genre_8_gender_pref,genre_9_gender_pref,genre_10_gender_pref,genre_11_gender_pref,genre_12_gender_pref,genre_13_gender_pref,genre_14_gender_pref,genre_15_gender_pref,genre_16_gender_pref,genre_17_gender_pref,genre_0_user_genre_interaction,genre_1_user_genre_interaction,genre_2_user_genre_interaction,genre_3_user_genre_interaction,genre_4_user_genre_interaction,genre_5_user_genre_interaction,genre_6_user_genre_interaction,genre_7_user_genre_interaction,genre_8_user_genre_interaction,genre_9_user_genre_interaction,genre_10_user_genre_interaction,genre_11_user_genre_interaction,genre_12_user_genre_interaction,genre_13_user_genre_interaction,genre_14_user_genre_interaction,genre_15_user_genre_interaction,genre_16_user_genre_interaction,genre_17_user_genre_interaction,genre_0_age_genre_pref,genre_1_age_genre_pref,genre_2_age_genre_pref,genre_3_age_genre_pref,genre_4_age_genre_pref,genre_5_age_genre_pref,genre_6_age_genre_pref,genre_7_age_genre_pref,genre_8_age_genre_pref,genre_9_age_genre_pref,genre_10_age_genre_pref,genre_11_age_genre_pref,genre_12_age_genre_pref,genre_13_age_genre_pref,genre_14_age_genre_pref,genre_15_age_genre_pref,genre_16_age_genre_pref,genre_17_age_genre_pref,item_avg_rating
0,4855,F,1,3253,4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,<25,1,4.162791,000000010000000000,40748,24372,10939,19090,85970,14732,1720,87655,7763,3730,12977,11992,8905,44876,24496,35907,12579,3113,5,5,15,16,10,1,0,18,2,0,0,11,0,5,3,1,2,0,50751,27086,11388,19061,72603,15267,1079,58644,8214,2317,15575,8229,6515,26072,29633,36268,11184,2894,3.483117
1,4855,F,1,3367,4,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,<25,2,4.162791,000000010000010000,40748,24372,10939,19090,85970,14732,1720,87655,7763,3730,12977,11992,8905,44876,24496,35907,12579,3113,5,5,15,16,10,1,0,18,2,0,0,11,0,5,3,1,2,0,50751,27086,11388,19061,72603,15267,1079,58644,8214,2317,15575,8229,6515,26072,29633,36268,11184,2894,3.582433
2,4855,F,1,2732,5,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,<25,2,4.162791,000010000000001000,40748,24372,10939,19090,85970,14732,1720,87655,7763,3730,12977,11992,8905,44876,24496,35907,12579,3113,5,5,15,16,10,1,0,18,2,0,0,11,0,5,3,1,2,0,50751,27086,11388,19061,72603,15267,1079,58644,8214,2317,15575,8229,6515,26072,29633,36268,11184,2894,3.999570
3,4855,F,1,2587,5,3,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,<25,3,4.162791,001100000001000000,40748,24372,10939,19090,85970,14732,1720,87655,7763,3730,12977,11992,8905,44876,24496,35907,12579,3113,5,5,15,16,10,1,0,18,2,0,0,11,0,5,3,1,2,0,50751,27086,11388,19061,72603,15267,1079,58644,8214,2317,15575,8229,6515,26072,29633,36268,11184,2894,3.747646
4,4855,F,1,3608,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,<25,1,4.162791,000000000000010000,40748,24372,10939,19090,85970,14732,1720,87655,7763,3730,12977,11992,8905,44876,24496,35907,12579,3113,5,5,15,16,10,1,0,18,2,0,0,11,0,5,3,1,2,0,50751,27086,11388,19061,72603,15267,1079,58644,8214,2317,15575,8229,6515,26072,29633,36268,11184,2894,3.254902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894144,4863,M,25,1889,5,335,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,25-45,2,3.607383,000010010000000000,189541,95453,27814,45574,232890,56593,5295,229551,24554,12558,55217,25187,26963,86934,116134,133546,48779,15385,35,15,4,4,89,23,8,173,3,10,15,9,9,39,28,37,18,2,139730,70921,22009,35601,190057,43017,4589,18

In [49]:
cat_features = ['user_id', 'gender', 'genre_0', 'genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5', 'genre_6', 'genre_7', 'genre_8', 'genre_9', 'genre_10', 'genre_11', 'genre_12', 'genre_13', 'genre_14', 'genre_15', 'genre_16', 'genre_17', 'age_group']
nums = ['total_genres_watched', 'user_avg_rating', 'genre_0_gender_pref', 'genre_1_gender_pref', 'genre_2_gender_pref', 'genre_3_gender_pref', 'genre_4_gender_pref', 'genre_5_gender_pref', 'genre_6_gender_pref', 'genre_7_gender_pref', 'genre_8_gender_pref', 'genre_9_gender_pref', 'genre_10_gender_pref', 'genre_11_gender_pref', 'genre_12_gender_pref', 'genre_13_gender_pref', 'genre_14_gender_pref', 'genre_15_gender_pref', 'genre_16_gender_pref', 'genre_17_gender_pref', 'genre_0_user_genre_interaction', 'genre_1_user_genre_interaction', 'genre_2_user_genre_interaction', 'genre_3_user_genre_interaction', 'genre_4_user_genre_interaction', 'genre_5_user_genre_interaction', 'genre_6_user_genre_interaction', 'genre_7_user_genre_interaction', 'genre_8_user_genre_interaction', 'genre_9_user_genre_interaction', 'genre_10_user_genre_interaction', 'genre_11_user_genre_interaction', 'genre_12_user_genre_interaction', 'genre_13_user_genre_interaction', 'genre_14_user_genre_interaction', 'genre_15_user_genre_interaction', 'genre_16_user_genre_interaction', 'genre_17_user_genre_interaction', 'genre_0_age_genre_pref', 'genre_1_age_genre_pref', 'genre_2_age_genre_pref', 'genre_3_age_genre_pref', 'genre_4_age_genre_pref', 'genre_5_age_genre_pref', 'genre_6_age_genre_pref', 'genre_7_age_genre_pref', 'genre_8_age_genre_pref', 'genre_9_age_genre_pref', 'genre_10_age_genre_pref', 'genre_11_age_genre_pref', 'genre_12_age_genre_pref', 'genre_13_age_genre_pref', 'genre_14_age_genre_pref', 'genre_15_age_genre_pref', 'genre_16_age_genre_pref', 'genre_17_age_genre_pref', 'item_avg_rating']

In [57]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool

# Список категориальных и числовых признаков
cat_features = ['item_id', 'user_id', 'gender', 'genre_0', 'genre_1', 'genre_2', 'genre_3',
                'genre_4', 'genre_5', 'genre_6', 'genre_7', 'genre_8', 'genre_9', 'genre_10',
                'genre_11', 'genre_12', 'genre_13', 'genre_14', 'genre_15', 'genre_16',
                'genre_17', 'age_group']

nums = ['total_genres_watched', 'user_avg_rating', 'genre_0_gender_pref', 'genre_1_gender_pref',
        'genre_2_gender_pref', 'genre_3_gender_pref', 'genre_4_gender_pref', 'genre_5_gender_pref',
        'genre_6_gender_pref', 'genre_7_gender_pref', 'genre_8_gender_pref', 'genre_9_gender_pref',
        'genre_10_gender_pref', 'genre_11_gender_pref', 'genre_12_gender_pref', 'genre_13_gender_pref',
        'genre_14_gender_pref', 'genre_15_gender_pref', 'genre_16_gender_pref', 'genre_17_gender_pref',
        'genre_0_user_genre_interaction', 'genre_1_user_genre_interaction', 'genre_2_user_genre_interaction',
        'genre_3_user_genre_interaction', 'genre_4_user_genre_interaction', 'genre_5_user_genre_interaction',
        'genre_6_user_genre_interaction', 'genre_7_user_genre_interaction', 'genre_8_user_genre_interaction',
        'genre_9_user_genre_interaction', 'genre_10_user_genre_interaction', 'genre_11_user_genre_interaction',
        'genre_12_user_genre_interaction', 'genre_13_user_genre_interaction', 'genre_14_user_genre_interaction',
        'genre_15_user_genre_interaction', 'genre_16_user_genre_interaction', 'genre_17_user_genre_interaction',
        'genre_0_age_genre_pref', 'genre_1_age_genre_pref', 'genre_2_age_genre_pref', 'genre_3_age_genre_pref',
        'genre_4_age_genre_pref', 'genre_5_age_genre_pref', 'genre_6_age_genre_pref', 'genre_7_age_genre_pref',
        'genre_8_age_genre_pref', 'genre_9_age_genre_pref', 'genre_10_age_genre_pref', 'genre_11_age_genre_pref',
        'genre_12_age_genre_pref', 'genre_13_age_genre_pref', 'genre_14_age_genre_pref', 'genre_15_age_genre_pref',
        'genre_16_age_genre_pref', 'genre_17_age_genre_pref', 'item_avg_rating']

user_df = df[['user_id', 'gender', 'age', 'age_group']].drop_duplicates()
item_df = df[['item_id', 'genre_0', 'genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5', 'genre_6',
              'genre_7', 'genre_8', 'genre_9', 'genre_10', 'genre_11', 'genre_12', 'genre_13',
              'genre_14', 'genre_15', 'genre_16', 'genre_17'] + nums].drop_duplicates()

# Преобразование категориальных признаков
df['gender'] = df['gender'].astype('category')

# Список жанровых столбцов
genre_cols = [col for col in df.columns if 'genre_' in col and not col.endswith('_pref') and
              not col.endswith('_interaction') and not col.endswith('_age_genre_pref')]

# Извлечение уникальных пользователей и фильмов
users = df['user_id'].unique()
items = df['item_id'].unique()

# Подготовка данных для обучения
X = df[cat_features + nums]
y = df['rating']

# Определение категориальных признаков для CatBoost (используем названия колонок)
# НЕ оборачиваем cat_features в дополнительный список
# cat_features = [cat_features]  # Удаляем или комментируем эту строку

# Инициализация и обучение модели CatBoost
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    cat_features=cat_features,  # Передаем список названий категориальных признаков
    verbose=100
)

train_pool = Pool(X, y, cat_features=cat_features)
model.fit(train_pool)

# Генерация рекомендаций для каждого пользователя
recommendations = []

for user in users:
    # Информация о пользователе
    user_info = user_df[user_df['user_id'] == user]
    user_gender = user_info['gender'].values[0]
    user_age = user_info['age'].values[0]
    user_age_group = user_info['age_group'].values[0]

    # Фильмы, которые пользователь уже смотрел
    items_seen = df[df['user_id'] == user]['item_id'].unique()

    # Фильмы для рекомендации
    items_to_recommend = np.setdiff1d(items, items_seen)
    items_to_recommend_df = item_df[item_df['item_id'].isin(items_to_recommend)]

    # Подготовка данных для предсказания
    user_features = pd.DataFrame({
        'user_id': [user] * len(items_to_recommend_df),
        'gender': [user_gender] * len(items_to_recommend_df),
        'age': [user_age] * len(items_to_recommend_df),
        'age_group': [user_age_group] * len(items_to_recommend_df)
    })

    predict_data = pd.concat([user_features.reset_index(drop=True),
                              items_to_recommend_df.reset_index(drop=True)], axis=1)

    # Убедимся, что все необходимые признаки присутствуют
    predict_data = predict_data[cat_features + nums]

    # Предсказание рейтингов
    predict_pool = Pool(predict_data, cat_features=cat_features)
    predictions = model.predict(predict_pool)
    predict_data['predicted_rating'] = predictions

    # Топ-10 рекомендаций
    top_10 = predict_data.sort_values('predicted_rating', ascending=False).head(10)
    recommendations.append({
        'user_id': user,
        'item_id': ' '.join(map(str, top_10['item_id'].tolist()))
    })

# Формирование итогового DataFrame с рекомендациями
recommendations_df = pd.DataFrame(recommendations)

# Вывод результатов
print(recommendations_df)


0:	learn: 1.1002525	total: 180ms	remaining: 3m
100:	learn: 0.9014762	total: 17.4s	remaining: 2m 34s
200:	learn: 0.8950518	total: 35.7s	remaining: 2m 22s
300:	learn: 0.8909966	total: 54.8s	remaining: 2m 7s
400:	learn: 0.8878343	total: 1m 14s	remaining: 1m 51s
500:	learn: 0.8855031	total: 1m 33s	remaining: 1m 32s
600:	learn: 0.8837504	total: 1m 52s	remaining: 1m 14s
700:	learn: 0.8821906	total: 2m 11s	remaining: 56s
800:	learn: 0.8807541	total: 2m 31s	remaining: 37.6s
900:	learn: 0.8794432	total: 2m 51s	remaining: 18.9s
999:	learn: 0.8782930	total: 3m 11s	remaining: 0us
      user_id                                            item_id
0        4855   1039 1583 1583 2789 1831 1583 1211 152 2784 2297
1        4065   3022 2789 2862 2365 2784 152 3022 2862 2210 1560
2        3331  2603 2789 3529 1831 2210 2784 1354 1560 1831 2603
3        5373   2789 2210 1354 2784 3418 1211 3694 1560 1831 779
4        2032       152 2365 3583 1039 2862 152 318 152 152 2365
...       ...                      

In [58]:
recommendations_df.to_csv('result.csv', index=False)

In [61]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool


# Преобразование категориальных признаков
df['gender'] = df['gender'].astype('category')

# Список жанровых столбцов
genre_cols = [col for col in df.columns if 'genre_' in col]

# Извлечение уникальных пользователей и фильмов
users = df['user_id'].unique()
items = df['item_id'].unique()

# Подготовка данных для обучения
X = df[['user_id', 'item_id', 'gender', 'age'] + genre_cols]
y = df['rating']

# Определение категориальных признаков для CatBoost
cat_features = ['user_id', 'item_id', 'gender']

# Инициализация и обучение модели CatBoost
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    cat_features=cat_features,
    verbose=100
)
train_pool = Pool(X, y, cat_features=cat_features)
model.fit(train_pool)

# Подготовка данных пользователей и фильмов
user_df = df[['user_id', 'gender', 'age']].drop_duplicates()
item_df = df[['item_id'] + genre_cols].drop_duplicates()

# Генерация рекомендаций для каждого пользователя
recommendations = []

for user in users:
    # Информация о пользователе
    user_info = user_df[user_df['user_id'] == user]
    user_gender = user_info['gender'].values[0]
    user_age = user_info['age'].values[0]
    
    # Фильмы, которые пользователь уже смотрел
    items_seen = df[df['user_id'] == user]['item_id'].unique()
    
    # Фильмы для рекомендации
    items_to_recommend = np.setdiff1d(items, items_seen)
    items_to_recommend_df = item_df[item_df['item_id'].isin(items_to_recommend)]
    
    # Подготовка данных для предсказания
    user_features = pd.DataFrame({
        'user_id': [user] * len(items_to_recommend_df),
        'gender': [user_gender] * len(items_to_recommend_df),
        'age': [user_age] * len(items_to_recommend_df)
    })
    predict_data = pd.concat([user_features.reset_index(drop=True), items_to_recommend_df.reset_index(drop=True)], axis=1)
    predict_data = predict_data[['user_id', 'item_id', 'gender', 'age'] + genre_cols]
    
    # Предсказание рейтингов
    predict_pool = Pool(predict_data, cat_features=cat_features)
    predictions = model.predict(predict_pool)
    predict_data['predicted_rating'] = predictions
    
    # Топ-10 рекомендаций
    top_10 = predict_data.sort_values('predicted_rating', ascending=False).head(10)
    recommendations.append({
        'user_id': user,
        'item_id': ' '.join(map(str, top_10['item_id'].tolist()))
    })

# Формирование итогового DataFrame с рекомендациями
recommendations_df = pd.DataFrame(recommendations)

# Вывод результатов
print(recommendations_df)


0:	learn: 1.1022056	total: 195ms	remaining: 3m 15s
100:	learn: 0.9326946	total: 14s	remaining: 2m 4s
200:	learn: 0.9256814	total: 31.9s	remaining: 2m 6s
300:	learn: 0.9213943	total: 50.1s	remaining: 1m 56s
400:	learn: 0.9181626	total: 1m 7s	remaining: 1m 41s
500:	learn: 0.9154352	total: 1m 25s	remaining: 1m 24s
600:	learn: 0.9132558	total: 1m 43s	remaining: 1m 8s
700:	learn: 0.9113388	total: 2m 2s	remaining: 52s
800:	learn: 0.9096034	total: 2m 20s	remaining: 34.8s
900:	learn: 0.9079881	total: 2m 38s	remaining: 17.4s
999:	learn: 0.9066112	total: 2m 56s	remaining: 0us
      user_id                                            item_id
0        4855             36 1583 152 36 1039 36 36 36 1543 2784
1        4065          36 3022 36 2564 3022 36 2862 36 3022 1638
2        3331                    36 36 36 36 36 2603 36 36 36 36
3        5373                      36 36 36 36 36 36 36 36 36 36
4        2032    2281 3022 3022 3022 3022 3022 1039 2784 36 3022
...       ...                        

In [23]:
import pandas as pd
from catboost import CatBoostRegressor, Pool

# Получаем список всех уникальных пользователей и фильмов
all_users = df['user_id'].unique()
all_items = df['item_id'].unique()

# Извлекаем уникальные характеристики пользователей
user_feature_columns = ['user_id', 'gender', 'age', 'age_group']
user_features = df[user_feature_columns].drop_duplicates('user_id')

# Извлекаем уникальные характеристики фильмов
item_feature_columns = [col for col in df.columns if col not in ['user_id', 'gender', 'age', 'age_group', 'rating', 'timestamp']]
item_features = df[['item_id'] + item_feature_columns].drop_duplicates('item_id')

# Удаляем дублирующиеся столбцы в item_features
item_features = item_features.loc[:, ~item_features.columns.duplicated()]

# Создаем пустой список для хранения рекомендаций
recommendations = []

# Определяем категориальные фичи
cat_features = ['user_id', 'item_id', 'gender', 'age_group']

# Получаем список признаков, использованных при обучении
feature_columns = model.feature_names_

# Проходим по каждому пользователю
for user in all_users:
    # Получаем фильмы, которые пользователь уже посмотрел
    watched_items = df[df['user_id'] == user]['item_id'].unique()
    
    # Определяем фильмы, которые пользователь еще не смотрел
    items_to_recommend = list(set(all_items) - set(watched_items))
    
    # Создаем DataFrame с парами пользователь-фильм для предсказания
    user_item_pairs = pd.DataFrame({'user_id': user, 'item_id': items_to_recommend})
    
    # Добавляем характеристики пользователя
    user_data = user_features[user_features['user_id'] == user]
    user_item_pairs = user_item_pairs.merge(user_data, on='user_id', how='left')
    
    # Добавляем характеристики фильмов с использованием параметра suffixes
    user_item_pairs = user_item_pairs.merge(item_features, on='item_id', how='left', suffixes=('', '_item'))
    
    # Проверяем наличие дублирующихся столбцов
    duplicates = user_item_pairs.columns[user_item_pairs.columns.duplicated()]
    if len(duplicates) > 0:
        print(f"Дублирующиеся столбцы для пользователя {user}: {duplicates}")
        # Удаляем дублирующиеся столбцы
        user_item_pairs = user_item_pairs.loc[:, ~user_item_pairs.columns.duplicated()]
    
    # Проверяем, какие признаки отсутствуют
    missing_cols = set(feature_columns) - set(user_item_pairs.columns)
    if missing_cols:
        # Для отсутствующих признаков заполним значениями по умолчанию (например, нулями)
        for col in missing_cols:
            user_item_pairs[col] = 0  # Или другое подходящее значение
    
    # Убедимся, что порядок столбцов соответствует тому, что использовался при обучении
    user_item_pairs = user_item_pairs[feature_columns]
    
    # Создаем объект Pool для данных предсказания
    prediction_data = Pool(data=user_item_pairs, cat_features=cat_features)
    
    # Предсказываем рейтинги
    preds = model.predict(prediction_data)
    user_item_pairs['predicted_rating'] = preds
    
    # Сортируем фильмы по предсказанному рейтингу
    top_10_items = user_item_pairs.sort_values(by='predicted_rating', ascending=False).head(10)
    
    # Формируем строку с топ-10 рекомендациями
    recommended_items = ' '.join(top_10_items['item_id'].astype(str).tolist())
    
    # Добавляем в список рекомендаций
    recommendations.append({'user_id': user, 'item_id': recommended_items})

# Создаем итоговый DataFrame
submission = pd.DataFrame(recommendations)

# Сохраняем в CSV файл
submission.to_csv('submission.csv', index=False)


TypeError: 'NoneType' object is not iterable

In [65]:
submission

,user_id,item_id
0,4855,2729 3072 2248 1104 3212 2735 631 1505 1575 1455
1,4065,2729 3072 2248 1104 896 1505 2862 2365 3212 631
2,3331,2729 3072 2248 3212 1104 651 2735 1505 631 460
3,5373,2729 3072 2248 460 1104 3443 3212 1505 2735 2862
4,2032,2729 3072 631 2365 2248 1104 896 2862 318 1505
...,...,...
6035,4455,2729 3072 631 896 2862 1505 2248 3443 460 2735
6036,5002,2729 3072 2248 3212 1104 2735 651 1575 896 1455
6037,4365,2729 3072 2248 3212 1104 896 1505 631 460 2735
6038,3700,2729 3072 1104 460 3212 1505 2365 3443 631 3062


In [49]:
a = list(df[df['user_id'] == 0]['item_id'].unique())

In [50]:
b =  [1039, 36, 1831, 2931, 2281, 2895, 3013, 2297, 2470, 1211]

In [60]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.neighbors import KNeighborsRegressor

# Список категориальных и числовых признаков
cat_features = ['item_id', 'user_id', 'gender', 'age_group'] + [f'genre_{i}' for i in range(18)]
nums = ['total_genres_watched', 'user_avg_rating'] + \
       [f'genre_{i}_gender_pref' for i in range(18)] + \
       [f'genre_{i}_user_genre_interaction' for i in range(18)] + \
       [f'genre_{i}_age_genre_pref' for i in range(18)] + \
       ['item_avg_rating']

# Объединяем все признаки
all_features = cat_features + nums

# Копируем исходный DataFrame для предобработки
df_processed = df.copy()

# Преобразуем категориальные признаки в df_processed
encoder = OrdinalEncoder()
df_processed[cat_features] = encoder.fit_transform(df_processed[cat_features])

# Подготовка данных для обучения
X = df_processed[all_features]
y = df_processed['rating']

# Инициализация и обучение модели KNN
model = KNeighborsRegressor(n_neighbors=5, algorithm='auto')
model.fit(X, y)

# Извлечение уникальных пользователей и фильмов
users = df['user_id'].unique()
items = df['item_id'].unique()

# Подготовка данных пользователей и фильмов
user_features_list = ['user_id', 'gender', 'age', 'age_group', 'total_genres_watched', 'user_avg_rating'] + \
                     [f'genre_{i}_gender_pref' for i in range(18)] + \
                     [f'genre_{i}_user_genre_interaction' for i in range(18)] + \
                     [f'genre_{i}_age_genre_pref' for i in range(18)]

item_features_list = ['item_id'] + [f'genre_{i}' for i in range(18)] + ['item_avg_rating']

user_df = df[user_features_list].drop_duplicates()
item_df = df[item_features_list].drop_duplicates()

# Преобразуем категориальные признаки в user_df и item_df
user_cat_features = [col for col in user_features_list if col in cat_features]
item_cat_features = [col for col in item_features_list if col in cat_features]

user_df_processed = user_df.copy()
item_df_processed = item_df.copy()

# Преобразуем категориальные признаки в user_df_processed
if user_cat_features:
    user_df_processed[user_cat_features] = encoder.transform(user_df_processed[user_cat_features])

# Преобразуем категориальные признаки в item_df_processed
if item_cat_features:
    item_df_processed[item_cat_features] = encoder.transform(item_df_processed[item_cat_features])

# Генерация рекомендаций для каждого пользователя
recommendations = []

for user in users:
    # Информация о пользователе
    user_info = user_df_processed[user_df_processed['user_id'] == user].iloc[0]
    
    # Фильмы, которые пользователь уже смотрел
    items_seen = df[df['user_id'] == user]['item_id'].unique()
    
    # Фильмы для рекомендации
    items_to_recommend = np.setdiff1d(items, items_seen)
    items_to_recommend_df = item_df_processed[item_df_processed['item_id'].isin(items_to_recommend)]
    
    # Подготовка данных для предсказания
    num_items = len(items_to_recommend_df)
    user_features = pd.DataFrame([user_info.values] * num_items, columns=user_info.index)
    
    predict_data = pd.concat([user_features.reset_index(drop=True), items_to_recommend_df.reset_index(drop=True)], axis=1)
    
    # Убираем дублирующиеся столбцы
    predict_data = predict_data.loc[:, ~predict_data.columns.duplicated()]
    
    # Используем те же признаки, что и при обучении
    predict_data = predict_data[all_features]
    
    # Предсказание рейтингов
    predictions = model.predict(predict_data)
    predict_data['predicted_rating'] = predictions
    
    # Топ-10 рекомендаций
    top_10 = predict_data.sort_values('predicted_rating', ascending=False).head(10)
    recommendations.append({
        'user_id': user,
        'item_id': ' '.join(map(str, top_10['item_id'].tolist()))
    })

# Формирование итогового DataFrame с рекомендациями
recommendations_df = pd.DataFrame(recommendations)

# Вывод результатов
print(recommendations_df)


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- genre_0
- genre_1
- genre_10
- genre_11
- genre_12
- ...
